In [40]:
import pandas as pd
import re as regex

from collections import Counter
import csv
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as pd

In [56]:
import sklearn

In [57]:
sklearn.__version__

'1.1.1'

In [3]:
## import Orthogroups without 70-15
og_no_70_15 = '../../pipeline_methods/Orthogroups.tsv'

In [4]:
og_no_70_15_df = pd.read_csv(og_no_70_15, dtype='string', sep='\t', index_col = 0)

In [5]:
# used to get the og a gene belongs to

df_lol = og_no_70_15_df.values.tolist()

og_dict_no_70_15 = {}

for i, row in enumerate(df_lol):
    og = og_no_70_15_df.index[i]
    for cell in row:
        if not pd.isnull(cell):
            for protein in cell.split(', '):
                og_dict_no_70_15[protein] = og

In [6]:
# use to get the group of genes assigned to an og

genes_per_og_no_70_15 = {}

for gene in og_dict_no_70_15:
    og = og_dict_no_70_15[gene]
    if og not in genes_per_og_no_70_15:
        genes_per_og_no_70_15[og] = []
    genes_per_og_no_70_15[og].append(gene)

In [7]:
genomes = [
'13FM-16-1',
'13FM-24-1',
'13FM-3-2',
'13FM-5-1',
'AG0004',
'AV1-1-1',
'BJ-90-611',
'BJ08-8',
'BN0123',
'BN0202',
'BN0252',
'CD0065',
'CD0142',
'CH0043',
'CH0072',
'CH0452',
'CH0461',
'CH1103',
'CH1164',
'CM0028',
'DB11-621',
'FJ0204804',
'FJ11SH-527-1',
'FJ12JN-084-3',
'FJ13SH05-2',
'FJ2000-62A',
'FJ2000-69A',
'FJ2001-112B',
'FJ2003-001A1',
'FJ2005113B',
'FJ2006-60A3',
'FJ72ZC7-77',
'FJ78-JJ',
'FJ81-JY',
'FJ81-MH',
'FJ81-SW',
'FJ81-ZP',
'FJ81221ZB11-14',
'FJ86-CT',
'FJ86061ZE3-39',
'FJ95054B',
'FJ95085AZB13-116',
'FJ98099',
'FJ99138',
'FJSH0703',
'GD06-53',
'GD08-2025',
'GY0040',
'HB-14',
'HB-LTH18',
'HN-0812-3',
'HN-158',
'HN10-1604',
'IA1',
'IB33',
'IB49',
'IC17',
'IE1K',
'IN0017',
'IN0054',
'IN0059',
'IN0114',
'IN0115',
'IN0116',
'IT0010',
'JS08-611',
'JS09-138',
'JX-09Z116-1',
'JX10-102',
'JX11-141',
'KA1-3-1',
'KA2-1-1',
'ML0060',
'ML0062',
'ML33',
'NG0012',
'NG0054',
'Nich-2-3-2',
'Nich-2-7-4',
'SC-10-120-65-2',
'SC-10-25-44-1',
'TG0004',
'TG0032',
'TN0001',
'TN0002',
'TN0050',
'TN0057',
'TN0090',
'TR0025',
'TW-1-1-1-B-1',
'TW-12CY-TB1-2',
'TW-12HL-DF1-2',
'TW-12HL-YL2-1',
'TW-12TD-RH1-1',
'TW-12TN-HB2-2',
'TW-12YL-DL3-2',
'TW-12YL-DP1-1',
'TW-12YL-TT4-1',
'TW-2-7-2-A-1',
'TW-6-2-2-B-1',
'TW-6-43-1',
'TW-CYBP1-3',
'TW-PT1-1',
'TW-PT3-1',
'TW-PT6-1',
'TW-TN4-2',
'US0041',
'VT0027',
'VT0030',
'WD-3-1',
'YN07205e',
'YN072310',
'YN072311',
'YN072313',
'YN08181e',
'YN08182c',
'YN126311',
'YN126441',
'YN8773-19',
'YN8773R-27',
'ZJ00-10',
'ZJ08-41',
'ZJ2011-7-1',
]

In [8]:
gene_count_per_og = {}

for og in genes_per_og_no_70_15:
    gene_count_per_og[og] = len(genes_per_og_no_70_15[og])

In [9]:
def regex_partition(content, separator):
    separator_match = regex.search(separator, content)
    if not separator_match:
        return content, content, content

    matched_separator = separator_match.group(0)
    parts = regex.split(matched_separator, content, 1)

    return parts[0], matched_separator, parts[1]

def trim_domain_names(input_list):
    regexp = '_[0-9]'
    trimmed_domains = []
    for i in range(len(input_list)):
        if len(regex_partition(input_list[i], regexp)[0]) > 2:
            trimmed_domains.append(regex_partition(input_list[i], regexp)[0])
        else:
            trimmed_domains.append(input_list[i])
    regexp = '_C|_N'
    trimmed_domains_noCN = []
    for i in range(len(trimmed_domains)):
        trimmed_domains_noCN.append(regex_partition(trimmed_domains[i], regexp)[0])
        if len(regex_partition(trimmed_domains[i], regexp)[0]) > 2:
            trimmed_domains_noCN.append(regex_partition(trimmed_domains[i], regexp)[0])
        else:
            trimmed_domains_noCN.append(trimmed_domains[i])
    regexp = 'Peptidase'
    trimmed_domains_noCN_peptidasegrouped = []
    for i in range(len(trimmed_domains_noCN)):
        trimmed_domains_noCN_peptidasegrouped.append(regex_partition(trimmed_domains_noCN[i], regexp)[1])
    trimmed_domains_noCN_peptidasegrouped_cased = []
    for i in range(len(trimmed_domains_noCN_peptidasegrouped)):
        trimmed_domains_noCN_peptidasegrouped_cased.append(trimmed_domains_noCN_peptidasegrouped[i].lower())
    return trimmed_domains_noCN_peptidasegrouped_cased

In [10]:
domain_counts_per_og = {}

for genome in genomes:
    kparse_out = "all_pfam_scan/"+ genome + ".pfamscan.kparse.out"
    with open(kparse_out, newline = '') as file:
        file_reader = csv.reader(file, delimiter = '\t')
        for row in file_reader:
            gene = row[0]
            if gene in og_dict_no_70_15: # some genes dont have ogs
                og = og_dict_no_70_15[gene]
            if og not in domain_counts_per_og:
                domain_counts_per_og[og] = {}
            domains_trimmed = trim_domain_names(row[1].split("~"))
            domains = list(set(domains_trimmed))
            for domain in domains:
                if domain in domain_counts_per_og[og]:
                    domain_counts_per_og[og][domain] += 1
                else:
                    domain_counts_per_og[og][domain] = 1

In [11]:
domains_per_og = {}

for og in domain_counts_per_og:
    domains_per_og[og] = []
    gene_count = gene_count_per_og[og]
    for domain in domain_counts_per_og[og]:
        domain_count = domain_counts_per_og[og][domain]
        if domain_count >= gene_count*0.5:
            domains_per_og[og].append(domain)

## some of them will be empty now
for og in domain_counts_per_og:
    if domains_per_og[og] == []:
        del domains_per_og[og]

In [12]:
for og in gene_count_per_og:
    if og not in domains_per_og:
        domains_per_og[og] = ['unannotated']

In [13]:
ogs_per_domain = {}

for og in domains_per_og:
    for domain in domains_per_og[og]:
        if domain not in ogs_per_domain:
            ogs_per_domain[domain] = 1
        else:
            ogs_per_domain[domain] += 1

In [14]:
identified_ogs_per_domain = {}

for og in domains_per_og:
    for domain in domains_per_og[og]:
        if domain not in identified_ogs_per_domain:
            identified_ogs_per_domain[domain] = [og]
        else:
            identified_ogs_per_domain[domain].append(og)

In [15]:
lineage_differentiating_ogs = []

with open('../pca_heat_map_phylogeny/lineage_differentiating_pavs.txt') as file:
    file_reader = csv.reader(file, delimiter = '\t')
    for row in file_reader:
        lineage_differentiating_ogs.append(row[0])

In [16]:
ratio = len(lineage_differentiating_ogs)/len(domains_per_og)

In [17]:
ogs_per_domain_expected = {}

for domain in ogs_per_domain:
    ogs_per_domain_expected[domain] = round(ogs_per_domain[domain] * ratio, 2)

In [18]:
ogs_per_domain_observed = {}

for og in lineage_differentiating_ogs:
    domains = domains_per_og[og]
    for domain in domains:
        if domain not in ogs_per_domain_observed:
            ogs_per_domain_observed[domain] = 1
        else:
            ogs_per_domain_observed[domain] += 1

for domain in ogs_per_domain:
    if domain not in ogs_per_domain_observed:
        ogs_per_domain_observed[domain] = 0

In [19]:
ogs_per_domain_notobserved = {}

for og in domains_per_og:
    if og not in lineage_differentiating_ogs:
        domains = domains_per_og[og]
        for domain in domains:
            if domain not in ogs_per_domain_notobserved:
                ogs_per_domain_notobserved[domain] = 1
            else:
                ogs_per_domain_notobserved[domain] += 1

for domain in ogs_per_domain:
    if domain not in ogs_per_domain_notobserved:
        ogs_per_domain_notobserved[domain] = 0

In [20]:
results_lol = []

for domain in ogs_per_domain_expected:
    observed = len(lineage_differentiating_ogs)
    not_observed = len(domains_per_og)-observed
    annotated = ogs_per_domain[domain]
    unannotated = len(domains_per_og) - annotated
    observed_annotated = ogs_per_domain_observed[domain]
    observed_unannotated = observed-observed_annotated
    notobserved_annotated = ogs_per_domain_notobserved[domain] 
    notobserved_unannotated = not_observed - notobserved_annotated
    table = [
        [observed_annotated, observed_unannotated],
        [notobserved_annotated, notobserved_unannotated]
    ]
    if (observed_annotated + observed_unannotated != observed or notobserved_annotated + notobserved_unannotated != not_observed or
        observed_annotated + notobserved_annotated != annotated or observed_unannotated + notobserved_unannotated != unannotated):
        print(observed)
        print(not_observed)
        print(annotated)
        print(unannotated)
        print(table[0])
        print(table[1])
        raise ValueError("sums arent right")
    p_value_fisher = stats.fisher_exact(table)[1]
    results_lol.append([domain, ogs_per_domain[domain], ogs_per_domain_expected[domain], ogs_per_domain_observed[domain], ogs_per_domain_observed[domain]/ogs_per_domain_expected[domain],p_value_fisher])

In [21]:
df = pd.DataFrame(results_lol, columns = ['domain', 'annotated','expected','observed', 'ratio','p_value_fisher'])

In [22]:
df = df[df.observed >= 3]

In [23]:
df = df[df.p_value_fisher <= 0.05]

In [24]:
df

,domain,annotated,expected,observed,ratio,p_value_fisher
5,kr,32,1.36,4,2.941176,4.519184e-02
9,ketoacyl-synt,37,1.57,5,3.184713,1.925612e-02
10,ps-dh,32,1.36,4,2.941176,4.519184e-02
11,acyl_transf,39,1.66,6,3.614458,5.651217e-03
115,polyprenyl_synt,13,0.55,4,7.272727,1.694794e-03
193,het,43,1.83,5,2.732240,3.460663e-02
621,tyrosinase,17,0.72,3,4.166667,3.324619e-02
918,aph,11,0.47,5,10.638298,5.073397e-05
3244,unannotated,6040,256.55,407,1.586435,2.081699e-37


In [25]:
df.to_csv("pfam_enrichment.txt", sep = '\t')

In [26]:
## NLRS stuff

In [27]:
df_unfiltered = pd.DataFrame(results_lol, columns = ['domain', 'annotated','expected','observed', 'ratio','p_value_fisher'])

In [28]:
df_unfiltered[df_unfiltered.domain=="nacht"]

,domain,annotated,expected,observed,ratio,p_value_fisher
363,nacht,21,0.89,3,3.370787,0.057425


In [29]:
df_unfiltered[df_unfiltered.domain=="nb-arc"]

,domain,annotated,expected,observed,ratio,p_value_fisher
492,nb-arc,6,0.25,1,4.0,0.229308


In [30]:
df_unfiltered[df_unfiltered.domain=="het"]

,domain,annotated,expected,observed,ratio,p_value_fisher
193,het,43,1.83,5,2.73224,0.034607


In [31]:
observed = len(lineage_differentiating_ogs)
not_observed = len(domains_per_og)-observed
annotated = ogs_per_domain['nb-arc'] + ogs_per_domain['nacht']
unannotated = len(domains_per_og) - annotated
observed_annotated = ogs_per_domain_observed['nb-arc'] + ogs_per_domain_observed['nacht']
observed_unannotated = observed-observed_annotated
notobserved_annotated = ogs_per_domain_notobserved['nb-arc'] + ogs_per_domain_notobserved['nacht'] 
notobserved_unannotated = not_observed - notobserved_annotated
table = [
    [observed_annotated, observed_unannotated],
    [notobserved_annotated, notobserved_unannotated]
]
if (observed_annotated + observed_unannotated != observed or notobserved_annotated + notobserved_unannotated != not_observed or
    observed_annotated + notobserved_annotated != annotated or observed_unannotated + notobserved_unannotated != unannotated):
    print(observed)
    print(not_observed)
    print(annotated)
    print(unannotated)
    print(table[0])
    print(table[1])
    raise ValueError("sums arent right")
print(table)
p_value_fisher = stats.fisher_exact(table)[1]

[[4, 583], [23, 13210]]


In [32]:
p_value_fisher

0.02601227111500909

In [33]:
expected = round(annotated * ratio, 2)
print(expected)

1.15


In [37]:
import scipy

In [38]:
print(scipy.__version__)

1.9.0
